In [30]:
import sys
sys.path.append('.') 
import pandas as pd
import h3
import os
from multiprocessing import Pool

In [5]:
import importlib
import src.SRFiltered_to_SimInput
importlib.reload(src.SRFiltered_to_SimInput)
import src.Simulation_Preparation
importlib.reload(src.Simulation_Preparation)
import src.Simulation_Mapper
importlib.reload(src.Simulation_Mapper)
import src.Simulation_PostProcessing
importlib.reload(src.Simulation_PostProcessing)
import src.Aggregated_Plots
importlib.reload(src.Aggregated_Plots)

<module 'src.Aggregated_Plots' from '/Users/hemingyi/Research/202410_skmob/TimeGeo_rebuild/src/Aggregated_Plots.py'>

In [6]:
import DT_user_param
print(DT_user_param.run_DT_simulation)

from src.SRFiltered_to_SimInput import decode_and_write_parameters, remove_redundant_stays, extract_frequent_users, extract_stay_regions_for_frequent_users, clean_and_format_fa_users
from src.Simulation_Preparation import generate_simulation_input, generate_simulation_parameters, split_simulation_inputs, activeness, otherLocations
from src.Simulation_Mapper import simulate
from src.Simulation_PostProcessing import compress_simulation_results
from src.Aggregated_Plots import plot_hourly_trip_counts, plot_dept_validation,  load_simulation_cdf, load_cdr_cdf, plot_stay_durations, parse_simulation_results, parse_observed_data, plot_trip_distance, plot_daily_visited_locations, plot_location_rank, count_time_periods, count_mode_transitions


<built-in method run_DT_simulation of PyCapsule object at 0x1103cbc60>


# 0-Data_Alightment

In [ ]:
def data_aligntment(df_path, mark):
    '''
    The data_alignment function expects your input Parquet table to include exactly these four columns:
	1.	caid - a unique identifier for each user (any hashable type); it will be internally turned into integer codes.
	2.	stay_start_timestamp - the timestamp (datetime or string) when the stay begins; this will be converted to UNIX seconds.
	3.	type - a label for the kind of stay (e.g. “home”, “work”, “other”); carried through verbatim into the output.
	4.	h3_id_region - an H3 index at resolution 16 (as an integer or string of digits); used to look up latitude/longitude.
    '''
    # 1) load
    df = pd.read_parquet(df_path)

    # 2) Convert H3 integer to hex string, get lat/lng
    df['h3_id_region_16'] = df['h3_id_region'].astype(int).apply(lambda x: format(x, 'x'))
    lat_lng = list(map(h3.h3_to_geo, df['h3_id_region_16']))
    df['Latitude'], df['Longitude'] = zip(*lat_lng)
    
    # 3) Convert timestamp to UNIX seconds
    df['timestamp'] = pd.to_datetime(df['stay_start_timestamp']).astype(int) // 10**9

    # 4) Insert constant zero column
    df['zero'] = 0

    # 5) Factorize user IDs
    df['caid'] = pd.factorize(df['caid'])[0]

    # 6) Reorder for export
    out_cols = ['caid', 'timestamp', 'type', 'zero', 'h3_id_region', 'Longitude', 'Latitude']
    out_path = f'./Data_CDR_to_SRFiltered/StayRegionsFiltered_{mark}.txt'
    df[out_cols].to_csv(out_path, sep=' ', index=False, header=False)

In [7]:
def ensure_dir(directory):
    '''
    Ensure that the given directory exists. If it does not, create it and log its creation.
    '''
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory created: {directory}")

ensure_dir("./results/Parameters")
ensure_dir("./results/Parameters/NonCommuters")
ensure_dir("./results/Parameters/Commuters")
ensure_dir("./results/SRFiltered_to_SimInput")
ensure_dir("./results/Simulation")
ensure_dir("./results/figs")

Directory created: ./results/Parameters
Directory created: ./results/Parameters/NonCommuters
Directory created: ./results/Parameters/Commuters
Directory created: ./results/SRFiltered_to_SimInput
Directory created: ./results/Simulation
Directory created: ./results/figs


In [8]:
mark = 'u5000' # test, u5000, full

# 2-Parameters

run_DT_simulation loads a sequence of stay‐region records for many individuals, computes their empirical daily/weekly activity patterns, and then simulates new trajectories under a Markov‐chain mobility model. It writes both the real and simulated distributions of inter‐stay times, daily location counts, and location sequences (home, work, others) into separate output files for “commuters” and “non-commuters.”

- commuter_mode (bool, default false): if true, only processes individuals who visit a “work” location; otherwise only those who don’t.
- min_num_stay (int, default 2): minimum number of recorded stays to include a person in the analysis.
- max_num_stay (int, default 3000): maximum number of stays to include a person (filters out very long records).
- nw_thres (double, default 1.0): minimum “normalized weekly active periods” required to include a person (filters very low-activity users).
- slot_interval (int, default 600): time granularity in seconds for discretizing the simulation timeline (e.g. 600 s = 10 min slots).
- rho (double, default 0.6): return‐probability weight in the preferential‐return step (higher → more likely to revisit known places).
- gamma (double, default –0.21): power‐law exponent shaping the decay of return probability with the number of distinct locations.

In [9]:
DT_user_param.run_DT_simulation(
    input_path=f"./Data_CDR_to_SRFiltered/StayRegionsFiltered_{mark}.txt",
    output_dir="./results/parameters",
    commuter_mode=False,
    min_num_stay=2,
    max_num_stay=3000,
    nw_thres=1.0,
    slot_interval=600,
    rho=0.6,
    gamma=-0.21
)

Total execution time for iterations: 14.8684 seconds


In [10]:
DT_user_param.run_DT_simulation(
    input_path=f"./Data_CDR_to_SRFiltered/StayRegionsFiltered_{mark}.txt",
    output_dir="./results/parameters",
    commuter_mode=True,
    min_num_stay=2,
    max_num_stay=3000,
    nw_thres=1.0,
    slot_interval=600,
    rho=0.6,
    gamma=-0.21
)

In [11]:
## 3-ParameterValues.py
#Decode indexed b1 and b2 values in commuter and non-commuter parameter files,replacing them with actual numeric values, and write the results to new files.
b1_array = list(range(1, 21))
b2_array = list(range(1, 21))

decode_and_write_parameters(
    b1_array=b1_array,
    b2_array=b2_array,
    commuter_input_path=    './results/Parameters/Commuters/ParametersCommuters.txt',
    noncommuter_input_path= './results/Parameters/NonCommuters/ParametersNonCommuters.txt',
    commuter_output_path=   './results/Parameters/ParametersCommuters.txt',
    noncommuter_output_path='./results/Parameters/ParametersNonCommuters.txt'
)

Decoded parameters written to:
  ./results/Parameters/ParametersCommuters.txt
  ./results/Parameters/ParametersNonCommuters.txt


# 3-SRFiltered_to_SimInput

In [12]:
## 0_removeRedundance.py
# This function removes consecutive stays at the same stay location from the input file.
# [Outputfile] FilteredStayRegions_set.txt: [0-userID, 1-timestamp, 2-trip_purpose, 3-location_ID(h3), 4-lon, 5-lat]

remove_redundant_stays(
    input_path=f'./Data_CDR_to_SRFiltered/StayRegionsFiltered_{mark}.txt',
    output_path='./results/SRFiltered_to_SimInput/FilteredStayRegions_set.txt')

In [13]:
## 1_FAUsers.py
# This function reads a whitespace-delimited stay-region file line by line, counts the number of distinct “stays” per user (incrementing whenever the location coordinates change), and writes out the IDs of all users whose stay count exceeds num_stays_threshold.
# num_stays_threshold (int, default=15): minimum number of distinct stays a user must have to be included.

extract_frequent_users(
    input_path='./results/SRFiltered_to_SimInput/FilteredStayRegions_set.txt', 
    output_path='./results/SRFiltered_to_SimInput/FAUsers.txt', 
    num_stays_threshold=15)

Saved 983 users to ./results/SRFiltered_to_SimInput/FAUsers.txt


In [14]:
## 2_FAUsers_StayRegions.py
# This function filters a master stay-region file to include only those records belonging to a predefined list of “frequent” users and writes each user’s unique stay-region entries (no consecutive duplicates) to a new file.
# [Outputfile] FAUsers_StayRegions.txt: [0-userID, 1-date, 2-time, 3-trip_purpose, 4-lon, 5-lat]

extract_stay_regions_for_frequent_users(
    fa_users_path='./results/SRFiltered_to_SimInput/FAUsers.txt',
    input_path='./results/SRFiltered_to_SimInput/FilteredStayRegions_set.txt',
    output_path='./results/SRFiltered_to_SimInput/FAUsers_StayRegions.txt'
)

Filtered stay regions written to: ./results/SRFiltered_to_SimInput/FAUsers_StayRegions.txt


In [15]:
## 3_Format_SimInput.py
# This function reads a pre-filtered stay-region file, re-indexes each user’s unique locations, maps numeric location types to letter codes, removes consecutive duplicate visits, and writes the cleaned, reformatted records to a new file.
# [Outputfile] FAUsers_Cleaned_Formatted.txt: [0-userID, 1-date, 2-time, 3-trip_purpose['o','h','w'], 4-lon, 5-lat, 6-Location_index]

clean_and_format_fa_users(
    input_path='./results/SRFiltered_to_SimInput/FAUsers_StayRegions.txt',
    output_path='./results/SRFiltered_to_SimInput/FAUsers_Cleaned_Formatted.txt'
)

Formatted data written to: ./results/SRFiltered_to_SimInput/FAUsers_Cleaned_Formatted.txt


# 5-Simulation

In [16]:
## 1_formatStays.py
#   This function is to generate input to be fed to the simulation script. The stays are available in the following format: [userId, Date, timestamp, locationType, lon, lat, locId]. The required input to the simulation is of the type: [locationId, frequency, lon, lat] with a separate line with userId to indicate switch of user.
# [Outputfile] simulation_location.txt: [locationId, frequency, lon, lat], with a separate line with userId to indicate switch of user

generate_simulation_input(
    input_path='./results/SRFiltered_to_SimInput/FAUsers_Cleaned_Formatted.txt',
    output_path='./results/Simulation/simulation_location.txt'
)

Simulation input written to: ./results/Simulation/simulation_location.txt


In [17]:
## 2_formatParameters.py
# This function builds a single flat file of “simulation‐ready” parameter records for both commuter and non-commuter users. It draws each user’s time‐of‐departure and trip‐duration from a Gaussian mixture (GMM) model, assigns a random “regular vs. flexible” label, and encodes their daily work schedule over a specified number of days.

generate_simulation_parameters( 
    commuter_path='./results/Parameters/ParametersCommuters.txt',
    noncommuter_path='./results/Parameters/ParametersNonCommuters.txt',
    output_path='./results/Simulation/simulation_parameter.txt',
    work_prob_weekday=0.829, # probability a commuter works on a weekday
    work_prob_weekend=0.354, # probability a commuter works on a weekend day
    num_days=1, # number of consecutive days to simulate (each of 144 ten-minute slots)
    reg_prob=0.846, # probability a commuter is labeled “regular” vs. “flexible”
    gmm_group_index=0) # gmm_group_index=0: Regular commuters, 1: Flexible commuters

# simulation_parameter.txt
# if work_flag == 1:
    # [user_index] [user_id] [work_flag] [home_tract] [work_tract] [reg]
    # [beta1] [beta2] [nw]
    # [working_d0] [ts_d0] [dur_d0]
    # [working_d1] [ts_d1] [dur_d1]
# elif work_flag == 0:
    # [user_index] [user_id] [work_flag] [home_tract] [work_tract] [reg]
    # [beta1] [beta2] [nw]
    # [working] [ts] [dur] = [0 0 0]

Generated simulation parameters at ./results/Simulation/simulation_parameter.txt


In [18]:
num_cpus=16

In [19]:
## 3_MapInputSynthesis.py
# Splits simulation_parameter.txt and simulation_location.txt into chunks by user ID for parallel processing (MapReduce-style).

split_simulation_inputs(
    parameter_path='./results/Simulation/simulation_parameter.txt',
    location_path='./results/Simulation/simulation_location.txt',
    formatted_user_path='./results/SRFiltered_to_SimInput/FAUsers_Cleaned_Formatted.txt',
    output_dir='./results/Simulation',
    num_cpus=num_cpus
)

Chunks saved in: ./results/Simulation


In [20]:
## 5_MapReduceInput/dataPreparation.py
# Prepare activeness.txt and otherlocation.txt for Mapper.py
activeness(
    noncomm_daily_path='./results/Parameters/NonComm_pt_daily.txt', 
    noncomm_weekly_path='./results/Parameters/NonComm_pt_weekly.txt', 
    comm_daily_path='./results/Parameters/Comm_pt_daily.txt', 
    comm_weekly_path='./results/Parameters/Comm_pt_weekly.txt', 
    output_path='./results/Simulation/activeness.txt'
    )
otherLocations(
    input_path='./results/Simulation/simulation_location.txt', 
    output_path='./results/Simulation/otherlocation.txt', 
    sample_fraction=0.02)


# of locations: 15481
# of users: 983


In [ ]:
## 5_MapReduceInput/Mapper.py
# This function runs a slot-by-slot, activity-driven mobility simulation for each user “shard” (indexed by file_index).

for file_index in range(num_cpus):
    simulate(
        file_index=file_index,
        other_locations_file='./results/Simulation/otherlocation.txt',
        activeness_file='./results/Simulation/activeness.txt',
        num_days=1,
        start_slot=0,
        users_locations_dir='./results/Simulation/Locations',
        users_parameters_dir='./results/Simulation/Parameters',
        output_dir='./results/Simulation/Mapped',
        #--log-normal + Cauchy break sampling--
        break_mean=3.75106,
        break_sd=0.681418,
        break_x0=0.0,
        break_gamma=0.1,
        #--preferential-return vs. exploration weights--
        rho=0.6,
        gamma_val=-0.21,
        #--minimum exploration distance and rank-selection exponent--
        min_dist=0.6,
        exponent=-0.86
    )

In [22]:
## 5_MapReduceInput/1_Compressor.py
# Compresses simulation result files in the specified folder that start with a given prefix.

compress_simulation_results(
    input_folder='./results/Simulation/Mapped/', 
    output_folder='./results/Simulation/Compressed/', 
    file_prefix='simulationResults_'
)

# Plots

In [23]:
# 5_MapReduceInput/AggregatedPlots/1-type_of_trip_hourly.py
plot_hourly_trip_counts(
    mapped_dir='./results/Simulation/Mapped/',
    output='./results/figs/1-HourlyTripCount.png'
)

HBW = [0, 0, 0, 1, 1, 22, 45, 104, 68, 34, 19, 11, 16, 31, 20, 57, 94, 78, 34, 12, 9, 6, 1, 5]
HBO = [113, 127, 130, 127, 128, 136, 120, 90, 61, 49, 37, 30, 35, 34, 57, 67, 90, 107, 112, 127, 144, 166, 135, 122]
NHB = [95, 108, 78, 99, 99, 101, 111, 93, 73, 40, 51, 85, 102, 52, 70, 100, 96, 113, 113, 97, 141, 116, 82, 110]
Total = [208, 235, 208, 227, 228, 259, 276, 287, 202, 123, 107, 126, 153, 117, 147, 224, 280, 298, 259, 236, 294, 288, 218, 237]
Number of users : 982
Number of commuter users : 442
Number of noncommuter users : 540


In [24]:
# 5_MapReduceInput/AggregatedPlots/2-StayDurations.py

# 1. Read and calculate PDF of simulation results
sim_cdf = load_simulation_cdf(
    mapped_dir='./results/Simulation/Mapped/',
    filename_pattern='simulationResults_',
    max_duration=36,
    resolution=6
)

# 2. Reads and calculates PDF of CDR data
cdr_cdf = load_cdr_cdf(
    cdr_file='./results/SRFiltered_to_SimInput/FAUsers_Cleaned_Formatted.txt',
    max_duration=36,
    resolution=6
)

# 3. Plotting
plot_stay_durations(
    sim_cdf=sim_cdf,
    cdr_cdf=cdr_cdf,
    output_file='./results/figs/2-StayDuration_All.png',
    xlim=(0, 24),
    ylim=(0.0001, 1),
    figsize=(4, 3)
)


Stay Durations
[0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333334, 1.0, 1.1666666666666667, 1.3333333333333333, 1.5, 1.6666666666666667, 1.8333333333333333, 2.0, 2.1666666666666665, 2.3333333333333335, 2.5, 2.6666666666666665, 2.8333333333333335, 3.0, 3.1666666666666665, 3.3333333333333335, 3.5, 3.6666666666666665, 3.8333333333333335, 4.0, 4.166666666666667, 4.333333333333333, 4.5, 4.666666666666667, 4.833333333333333, 5.0, 5.166666666666667, 5.333333333333333, 5.5, 5.666666666666667, 5.833333333333333, 6.0, 6.166666666666667, 6.333333333333333, 6.5, 6.666666666666667, 6.833333333333333, 7.0, 7.166666666666667, 7.333333333333333, 7.5, 7.666666666666667, 7.833333333333333, 8.0, 8.166666666666666, 8.333333333333334, 8.5, 8.666666666666666, 8.833333333333334, 9.0, 9.166666666666666, 9.333333333333334, 9.5, 9.666666666666666, 9.833333333333334, 10.0, 10.166666666666666, 10.333333333333334, 10.5, 10.666666666666666, 10.833333333333334, 11.0, 11.16666666666

In [25]:
# 5_MapReduceInput/AggregatedPlots/3-DurationDist.py

mapped_dir = './results/Simulation/Mapped/'
observed_file = './results/SRFiltered_to_SimInput/FAUsers_Cleaned_Formatted.txt'

usersDailyLocationCount, usersTripDistances = parse_simulation_results(mapped_dir)
user1ObsLocationCount, user1ObsTripDistances = parse_observed_data(observed_file)
plot_trip_distance(usersTripDistances, user1ObsTripDistances, './results/figs/3-TripDistance.png')

Trip Distance
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 114, 115, 118, 119, 120, 121, 122, 124, 125, 126, 127, 130, 131, 134, 135, 136, 137, 138, 139, 141, 142, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 161, 162, 164, 165, 166, 167, 169, 170, 171, 172, 173, 175, 176, 177, 178, 181, 182, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 200, 204, 206, 207, 208, 211, 212, 214, 215, 216, 219, 220, 221, 222, 226, 227, 228, 229, 232, 233, 236, 237, 240, 243, 246, 247, 248, 250, 251, 253, 256, 263, 264, 265, 266, 267, 269, 270, 271, 272, 273, 274, 278, 281, 282

In [26]:
# 5_MapReduceInput/AggregatedPlots/4-VisitedLocCount.py

plot_daily_visited_locations(
    mapped_dir='./results/Simulation/Mapped/',
    obs_input_path='./results/SRFiltered_to_SimInput/FAUsers_Cleaned_Formatted.txt',
    output_path='./results/figs/4-numVisitedLocations'
)

Daily Visited locations
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16] [0.012219959266802444, 0.14765784114052954, 0.2637474541751528, 0.22912423625254583, 0.14663951120162932, 0.09266802443991853, 0.045824847250509164, 0.023421588594704685, 0.012219959266802444, 0.012219959266802444, 0.0050916496945010185, 0.004073319755600814, 0.003054989816700611, 0.0010183299389002036, 0.0010183299389002036]


In [27]:
# 5_MapReduceInput/AggregatedPlots/5-frequency.py

plot_location_rank(
    mapped_dir='./results/Simulation/Mapped/',
    sim_input_file='./results/SRFiltered_to_SimInput/FAUsers_Cleaned_Formatted.txt',
    output_file='./results/figs/5-LocationRank-User1.png',
    top_n=50)

In [28]:
# 5_MapReduceInput/AggregatedPlots/6-TimeOfDay.py
count_time_periods(mapped_dir='./results/Simulation/Mapped/')

AM = 591
PM = 728
MD = 557
RD = 1555
Total = 3431.0
0.17225298746721074
0.21218303701544738
0.1623433401340717
0.4532206353832702


In [29]:
# 5_MapReduceInput/AggregatedPlots/7-TypeOfTrip.py
count_mode_transitions(mapped_dir='./results/Simulation/Mapped/')

HBW = 669
HBO = 2365
NHB = 397
Total = 3431
